
# Final Project: Tesla vs GameStop — Stock Price and Revenue Dashboard

**Autor:** _Completa con tu nombre_  
**Curso:** IBM Data Science / Python Project for Data Science

Objetivo: Extraer datos de precio (yfinance) y de ingresos (web‑scraping) para **Tesla (TSLA)** y **GameStop (GME)**, luego construir tableros para comparar **precio** vs **ingresos**.

> Requisitos: `yfinance`, `pandas`, `requests`, `beautifulsoup4`, `lxml`, `plotly`.


In [1]:

# === 0) Dependencias ===
# Ejecuta esta celda si faltan paquetes
# Nota: en algunos entornos necesitarás reiniciar el kernel tras la instalación.
try:
    import yfinance, pandas, bs4, lxml, plotly
except Exception:
    %pip -q install yfinance pandas requests beautifulsoup4 lxml plotly


In [2]:

# === 1) Imports y utilidades ===
import re
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import io

pd.options.display.float_format = '{:,.6f}'.format


In [3]:
def _clean_money_to_float(x: str) -> float:
    if x is None: return float('nan')
    s = str(x).strip()
    if s in {"", "-", "—", "N/A"}: return float('nan')
    s = re.sub(r'[^0-9.\-]', '', s)
    try: return float(s)
    except ValueError: return float('nan')

def _from_yfinance_quarterly_revenue(symbol: str) -> pd.DataFrame:
    import yfinance as yf
    t = yf.Ticker(symbol)

    s = None
    # 1) Intento en quarterly_financials
    qfin = getattr(t, "quarterly_financials", None)
    if qfin is not None and not qfin.empty and "Total Revenue" in qfin.index:
        s = qfin.loc["Total Revenue"].dropna()

    # 2) Fallback en quarterly_income_stmt (algunas versiones lo exponen así)
    if s is None or s.empty:
        qinc = getattr(t, "quarterly_income_stmt", None)
        if qinc is not None and not qinc.empty and "Total Revenue" in qinc.index:
            s = qinc.loc["Total Revenue"].dropna()

    if s is None or s.empty:
        raise RuntimeError(f"yfinance no tiene 'Total Revenue' para {symbol}")

    # Asegura nombres estándar
    df = (
        s.rename("Revenue")              # <— fuerza el nombre de la Serie
         .rename_axis("Date")            # <— fuerza el nombre del índice
         .reset_index()                  # => columnas: ['Date','Revenue']
    )
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["Revenue"] = pd.to_numeric(df["Revenue"], errors="coerce")
    df = df.dropna(subset=["Date","Revenue"]).sort_values("Date").reset_index(drop=True)
    return df


def scrape_quarterly_revenue(symbol: str, company_slug: str) -> pd.DataFrame:
    """
    Intenta 2 estrategias en Macrotrends. Si no encuentra tabla válida, recurre a yfinance.
    Devuelve ['Date','Revenue'] ordenado ascendente.
    """
    import re, pandas as pd, requests
    from bs4 import BeautifulSoup

    def _clean_money_to_float(x: str) -> float:
        if x is None: return float('nan')
        s = str(x).strip()
        if s in {"", "-", "—", "N/A"}: return float('nan')
        s = re.sub(r'[^0-9.\-]', '', s)
        try: return float(s)
        except ValueError: return float('nan')

    url = f"https://www.macrotrends.net/stocks/charts/{symbol}/{company_slug}/revenue"
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "no-cache", "Pragma": "no-cache",
    }

    try:
        html = requests.get(url, headers=headers, timeout=30)
        html.raise_for_status()
        soup = BeautifulSoup(html.text, "lxml")

        # Camino A: encabezado "Quarterly Revenue" -> siguiente tabla
        header = soup.find(lambda tag: tag.name in ("h2","h3") and "Quarterly Revenue" in tag.get_text(strip=True))
        table = header.find_next("table") if header else None
        cand = pd.read_html(str(table))[0] if table else None

        # Camino B: escanear tablas y elegir la que tenga fecha + revenue
        if cand is None or cand.empty:
            for tbl in soup.find_all("table"):
                try:
                    tdf = pd.read_html(io.StringIO(str(tbl)))[0]
                except Exception:
                    continue
                cols = [str(c).lower() for c in tdf.columns.astype(str).tolist()]
                if any("revenue" in c for c in cols) and any(c in {"date","quarter","quarter end","quarter ended"} for c in cols):
                    cand = tdf
                    break

        if cand is None or cand.empty:
            raise RuntimeError("No se encontró tabla de ingresos trimestrales en Macrotrends.")

        # Normaliza
        rename_map = {}
        for c in cand.columns:
            cl = str(c).strip().lower()
            if cl in {"date","quarter","quarter end","quarter ended"}: rename_map[c] = "Date"
            if "revenue" in cl: rename_map[c] = "Revenue"
        cand = cand.rename(columns=rename_map)

        keep = [c for c in ["Date","Revenue"] if c in cand.columns]
        df = cand[keep].dropna(how="all").copy()
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df["Revenue"] = df["Revenue"].apply(_clean_money_to_float)
        df = df.dropna(subset=["Date","Revenue"]).sort_values("Date").reset_index(drop=True)

        if len(df) < 8:
            raise RuntimeError("Tabla encontrada, pero parece incompleta.")

        return df

    except Exception as e:
        print(f"[Aviso] Macrotrends falló para {symbol}: {e}. Usando yfinance como respaldo.")
        return _from_yfinance_quarterly_revenue(symbol)



def get_stock_history(ticker: str, period: str = "max") -> pd.DataFrame:
    """
    Descarga historial de precios de cierre para un ticker con yfinance.
    Devuelve un DataFrame con columna 'Date' y demás precios.
    """
    tk = yf.Ticker(ticker)
    df = tk.history(period=period).reset_index()
    if 'date' in df.columns and 'Date' not in df.columns:
        df = df.rename(columns={'date': 'Date'})
    df['Date'] = pd.to_datetime(df['Date'])
    return df

def make_dashboard(stock_df, revenue_df, title_prefix: str):
    """
    Entradas:
      stock_df  -> DataFrame con columnas ['Date','Close']
      revenue_df-> DataFrame con columnas ['Date','Revenue']
      title_prefix -> texto para títulos
    Salida: objeto Figure de Plotly
    """
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=False,
        subplot_titles=(f"{title_prefix} — Precio de Cierre",
                        f"{title_prefix} — Ingresos Trimestrales (USD)")
    )

    # Línea de precio
    fig.add_trace(
        go.Scatter(x=stock_df["Date"], y=stock_df["Close"], mode="lines", name="Close"),
        row=1, col=1
    )

    # Barras de ingresos
    fig.add_trace(
        go.Bar(x=revenue_df["Date"], y=revenue_df["Revenue"], name="Revenue"),
        row=2, col=1
    )

    fig.update_layout(
        height=800, width=1000,
        title_text=f"{title_prefix}: Precio vs Ingresos",
        showlegend=True
    )
    return fig


## Pregunta 1 — Extraer datos de **Tesla** con yfinance _(2 puntos)_


In [4]:
# Q1
tesla_ticker = "TSLA"
tesla_data = get_stock_history(tesla_ticker, period="max")
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.000000,0.000000
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.000000,0.000000
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.000000,0.000000
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.000000,0.000000
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.000000,0.000000



## Pregunta 2 — Extraer **ingresos de Tesla** con web‑scraping _(1 punto)_
Origen recomendado: **Macrotrends**.


In [5]:
# Q2
tesla_revenue = scrape_quarterly_revenue("TSLA", "tesla")
tesla_revenue.tail()


[Aviso] Macrotrends falló para TSLA: No se encontró tabla de ingresos trimestrales en Macrotrends.. Usando yfinance como respaldo.


,Date,Revenue
0,2024-06-30,"25,500,000,000.000000"
1,2024-09-30,"25,182,000,000.000000"
2,2024-12-31,"25,707,000,000.000000"
3,2025-03-31,"19,335,000,000.000000"
4,2025-06-30,"22,496,000,000.000000"



## Pregunta 3 — Extraer datos de **GameStop** con yfinance _(2 puntos)_


In [6]:
# Q3
gme_ticker = "GME"
gme_data = get_stock_history(gme_ticker, period="max")
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.000000,0.000000
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.000000,0.000000
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.000000,0.000000
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.000000,0.000000
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.000000,0.000000



## Pregunta 4 — Extraer **ingresos de GameStop** con web‑scraping _(1 punto)_


In [7]:
# Q4
gme_revenue = scrape_quarterly_revenue("GME", "gamestop")
gme_revenue.tail()

[Aviso] Macrotrends falló para GME: No se encontró tabla de ingresos trimestrales en Macrotrends.. Usando yfinance como respaldo.


,Date,Revenue
0,2024-04-30,"881,800,000.000000"
1,2024-07-31,"798,300,000.000000"
2,2024-10-31,"860,300,000.000000"
3,2025-01-31,"1,282,600,000.000000"
4,2025-04-30,"732,400,000.000000"



### Nota sobre limpieza de fechas y montos
- `Date` se convierte con `pd.to_datetime(..., errors="coerce")` para evitar errores por formatos mixtos.  
- `Revenue` se limpia con `_clean_money_to_float`, que elimina símbolos y comas.



## Pregunta 5 — Tablero **Tesla**: Precio vs Ingresos _(2 puntos)_


In [8]:
# Q5
fig_tsla = make_dashboard(tesla_data[["Date","Close"]], tesla_revenue[["Date","Revenue"]], "Tesla (TSLA)")
fig_tsla.show()
# Opcional: guarda el tablero como HTML para adjuntar al envío
fig_tsla.write_html("tesla_dashboard.html")
print("Guardado: tesla_dashboard.html")

Guardado: tesla_dashboard.html



## Pregunta 6 — Tablero **GameStop**: Precio vs Ingresos _(2 puntos)_


In [9]:

# Q6
fig_gme = make_dashboard(gme_data[["Date","Close"]], gme_revenue[["Date","Revenue"]], "GameStop (GME)")
fig_gme.show()
# Opcional: guarda el tablero como HTML
fig_gme.write_html("gamestop_dashboard.html")
print("Guardado: gamestop_dashboard.html")


Guardado: gamestop_dashboard.html



## Pregunta 7 — Compartir tu Notebook _(2 puntos)_
1. Sube este `.ipynb` a GitHub o compártelo en IBM Skills Network/Colab.  
2. Adjunta capturas de:
   - Head de los DataFrames en Preguntas 1–4.
   - Los dos tableros.
3. Incluye los archivos HTML exportados si corresponde.



## Depuración rápida

- **`ValueError` al convertir fechas**: Usa `pd.to_datetime(col, errors="coerce")` y revisa valores nulos.  
- **Cambios en Macrotrends**: Si cambian los encabezados, ajusta el mapeo de columnas en `scrape_quarterly_revenue`.  
- **Bloqueos de red**: Ejecuta en Colab o un entorno con salida a internet.
